<img src="https://gitlab.version.fz-juelich.de/grosch1/pvlink_demo_for_jupytercon/-/raw/JupyterCon2020/img/logo.png"
     style="float:right; width:25%; height:25%; margin-top:25px;">

# Jupyter for interactive In-Situ Visualization with ParaView/Catalyst

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/git/https%3A%2F%2Fgitlab.version.fz-juelich.de%2Fgrosch1%2Fpvlink_demo_for_jupytercon/JupyterCon2020)  
Try out the *pvlink* extension and the example notebooks in binder.

# Behind the Scenes

To checkout how *pvlink* works and what happens behind the scenes, [click here](https://gitlab.version.fz-juelich.de/grosch1/pvlink_demo_for_jupytercon/-/blob/JupyterCon2020/BehindTheScenes.md).

# Basic Usage

> If you are trying this notebook out on Binder, you need to use [jupyter server proxy](#Using-jupyter-server-proxy).

## SimpleRenderer

Start a ParaViewWeb server on the command line, for example using the code from the [official ParaView example](https://kitware.github.io/paraviewweb/examples/RemoteRenderer.html#Using-ParaView-as-server).

In [ ]:
%%script bash --bg
pvpython pv_server.py --port 1234 --authKey wslink-secret

You can then connect to the ParaViewWeb Server using a `SimpleRenderer`, passing the session URL and the authentication key.

In [ ]:
from pvlink import SimpleRenderer

simple = SimpleRenderer(sessionURL='ws://localhost:1234/ws', authKey='wslink-secret')
display(simple)

> If you are trying this notebook out on Binder, you need to use [jupyter server proxy](#Using-jupyter-server-proxy).

Note that in this setup, it is not possible to change the content to be rendered from within the notebook. The content is determined only at the start of the ParaViewWeb server.

## RemoteRenderer

Most likely you *do* want to be able to make changes to the displayed source. In this case you should use the `RemoteRenderer`, which handles the ParaViewWeb server for you. You can still pass all arguments of the webserver as python arguments, in both short and long form.

In [ ]:
from pvlink import RemoteRenderer

In [ ]:
print(RemoteRenderer.webserver_arguments_help())

For example, `active_renderer = RemoteRenderer(port=8080)` and `active_renderer = RemoteRenderer(p=8080)` would both set the server to start on port 8080.

You can also pass no arguments. In that case, the next free port starting from 8080 will be chosen and a random authKey will be generated for you.

### Using RemoteRenderer

> If you are trying this notebook out on Binder, you need to use [jupyter server proxy](#Using-jupyter-server-proxy).

In [ ]:
renderer = RemoteRenderer()
display(renderer)

In [ ]:
print('renderer port:\t', renderer.port)
print('renderer authKey:', renderer.authKey)

The window that appears will be blank since there are no views or sources. These can be created using ParaView's python module.

In [ ]:
from paraview import simple
from pvlink.utility import SetRecommendedRenderSettings

# Create a view and...
view1 = simple.CreateView('RenderView', 'view1')
# ...disable interactor-based render calls and
# ensure pvserver-side rendering (if applicable)
SetRecommendedRenderSettings(view1)
# Create and show a source
source1 = simple.Cone()
simple.Show(source1, view1)

In [ ]:
# Update the renderer widget to display the changes
renderer.update_render()

### Multiple views

You can create more than one view in ParaView. Per default, a `RemoteRenderer`will show the active view.

In [ ]:
print(renderer.viewID) # viewID = -1 will always show active view

In [ ]:
# Create a second view
view2 = simple.CreateView('RenderView', 'view2')
view2.Background = [0,0,0]
SetRecommendedRenderSettings(view2)
# And show a different source in it
source2 = simple.Sphere()
simple.Show(source2, view2)

The renderer will now show the second view, since it is the currently active one.

In [ ]:
renderer.update_render()
display(renderer)

If you want a `RemoteRenderer` to show a fixed view, regardless of which view is active, you can do so by setting its viewID.

In [ ]:
renderer.viewID = view2.GetGlobalIDAsString()

Changing the active view will now not change the view in the `RemoteRenderer` anymore.

In [ ]:
simple.SetActiveView(view1)
renderer.update_render()
display(renderer)

## Using jupyter server proxy

`pvlink` supports the usage of [jupyter server proxy](https://github.com/jupyterhub/jupyter-server-proxy). If your notebook is not running locally and you cannot or do not want to tunnel the ParaViewWeb server to your local workstation, you should use *jupyter server proxy*.

To do so, you need to specify the baseURL behind which your notebook is running in the baseURL initialization argument of your `RemoteRenderer`.
> *Example:* If your notebook is running at http://localhost:8888 and you would access a process running on port 1234 via http://localhost:8888/proxy/1234, the baseURL would be the part before "proxy", localhost:8888.

If your notebook is running on HTTPS, set `useJupyterServerProxyHttps=True` to use the HTTPS certificates of the notebook server. This is necessary and keeps the connection between the client browser and the (jupyter server) proxy encrypted.
You also need to set `disableExternalPort=True` in order to disable port checking during the WebSocket opening handshake. Otherwise, the connection will fail with a port mismatch error as the notebook and the webserver do not run on the same port.

For example, for a notebook running on https://jupyter-jsc.fz-juelich.de, the `RemoteRenderer` would be setup like this:
```proxied_renderer = RemoteRenderer(baseURL='jupyter-jsc.fz-juelich.de/user/<user>/<workspace>', 
                                  disableExternalPort=True,
                                  useJupyterServerProxyHttps=True)```

In [ ]:
# You will most likely have to change the baseURL to the URL at which your notebook server is running
proxied_renderer = RemoteRenderer(baseURL='localhost:8888', 
                                  disableExternalPort=True,
                                  useJupyterServerProxy=True) # useJupyterServerProxyHttps if https

For more information on how to use the `RemoteRenderer`, see the [RemoteRenderer section](#RemoteRenderer) of this notebook.

In [ ]:
from paraview import simple
from pvlink.utility import SetRecommendedRenderSettings

# Create a view and...
view3 = simple.CreateView('RenderView', 'view3')
# ...disable interactor-based render calls and
# ensure pvserver-side rendering (if applicable)
SetRecommendedRenderSettings(view3)
# Create and show a source
source3 = simple.Box()
simple.Show(source3, view3)

In [ ]:
proxied_renderer.viewID = view3.GetGlobalIDAsString()
display(proxied_renderer)